In [ ]:
import flopy
import numpy as np
import matplotlib.pyplot as plt
import shutil
from pathlib import Path

from datetime import datetime
from tempfile import TemporaryDirectory
from tempfile import mkdtemp

import os
import platform
import subprocess
from shapely.geometry import Polygon, Point
from scipy.interpolate import make_interp_spline

In [ ]:
def locate_ground_surface_lay(row=150):
    cross_section = ibound[:,:,0]
    lay = np.where(cross_section[:,row]==1)[0][0]
    return lay

def meter_to_col(col_meter):
    return int(np.round(col_meter/delc,0))

def meter_to_row(row_meter):
    return int(np.round(row_meter/delc,0))

def meter_to_lay(lay_meter):
    return int(np.round((top_elev-lay_meter)/layer_thickness,0))

def col_to_meter(col):
    return col*delc

def row_to_meter(row):
    return row*delc

def lay_to_meter(row):
    return 190.0-row*layer_thickness

In [ ]:

def change_array_within_polygon(array, row_coords, col_coords, value):

    # Create a polygon object from the row and column coordinates
    polygon = Polygon(zip(col_coords, row_coords))

    # Create a boolean mask indicating which points fall within the polygon
    mask = np.zeros_like(array, dtype=bool)
    for row in range(array.shape[0]):
        for col in range(array.shape[1]):
            point = Point(col, row)
            if polygon.contains(point) or polygon.intersects(point):
                mask[row, col] = True

    # Set the values of the array that fall within the polygon to the new value
    array[mask] = value

    return array


# Workspace

In [ ]:
# Define model workspace

myt = datetime.now().strftime('%m%d%H%M%S')
name = 'wasaga'
temp_dir = mkdtemp(prefix='_T{}_{}_'.format(myt,name))
os.makedirs(Path(temp_dir) / '_output')

#model_ws = "./modflow_model_2"
model_ws = temp_dir

def open_file(path):
    if platform.system() == "Windows":
        os.startfile(path)
    elif platform.system() == "Darwin":
        subprocess.Popen(["open", path])
    else:
        subprocess.Popen(["xdg-open", path])

open_file(model_ws)

In [ ]:
model_ws

In [ ]:
# base
parms= {}
parms['sp1']= 0.0002
parms['hk']= 6.9
parms['al'] = 5
base_parms = parms

# Model

In [ ]:
# Create a MODFLOW model instance
mf = flopy.modflow.Modflow(modelname="example_model",version='mf2005',
                              exe_name='mf2005', 
                              model_ws=model_ws)

In [ ]:
# # Create a MODFLOW model instance
# model = flopy.seawat.Seawat(modelname="example_model",
#                             exe_name='swtv4', 
#                               model_ws=model_ws)


# DIS

In [ ]:
# Define model dimensions
nlay = int(60 * 2)
nrow = int(220 * 2)
ncol = int(220 * 2)
delr = 2200/nrow  # Cell width along rows (extend 2200 / 220)
delc = 2200/ncol  # Cell width along columns (extend 2200 / 220)

# Define top elevation
layer_thickness = 30.0 / nlay

top_elev = 190.0
botm_elev = [top_elev - (i * layer_thickness) for i in range(1, nlay + 1)]
# Define layer thickness (uniform thickness)


perlen = 365
nper = int(10)
#nper = int(20)

steady = np.append(np.array([True]),np.repeat(False,nper-1))
#steady=[False,False,True]
#steady = True

# Create DIS package



In [ ]:
dis = flopy.modflow.ModflowDis(mf, nlay=nlay, nrow=nrow, ncol=ncol, 
                               delr=delr, delc=delc,top=top_elev, 
                               botm=botm_elev,
                               nper = nper, perlen = perlen,
                               nstp = 1, steady = steady)


# Ibound

In [ ]:


x = np.array([0, 100, 340, 600, 750, 850, 1100, 1350, 1450, 1530, 1650, 1750, 1850, 2050, 2200])
y_topo = np.array([190,190,187,184,184,184,183,182,179,175.5,175,173.8,173.5,172,171])#topo
y_clay = np.array([185,184,180,175,174,174,173,170,169,168.5,167.5,166.8,166.5,165,164]) #clay


# Interpolate the data using smoothing splines
spl = make_interp_spline(x, y_topo)
x_smooth_topo = np.linspace(x.min(), x.max(), 30)
y_smooth_topo = spl(x_smooth_topo)

col_coords = (x_smooth_topo/delr).round(0)
col_coords_topo = list(col_coords) +[nrow,0]

row_coords = ((top_elev - y_smooth_topo)/layer_thickness).round(0)
row_coords_topo  =  list(row_coords) +[0,0]



# Interpolate the data using smoothing splines
spl = make_interp_spline(x, y_clay)
x_smooth = np.linspace(x.min(), x.max(), 30)
y_smooth = spl(x_smooth)

col_coords = (x_smooth/delr).round(0)
col_coords_clay = list(col_coords) +[nrow,0]
row_coords = ((top_elev - y_smooth)/layer_thickness).round(0)
row_coords_clay  =  list(row_coords) +[nlay,nlay]


# Ibound & Strt

In [ ]:

# Define ibound and initial conditionsa
ibound = np.ones((nlay, nrow, ncol), dtype=int)
initial_head = np.full((nlay, nrow, ncol), 200.0, dtype=float)

# Set constant head boundary conditions for the first and last rows
array = change_array_within_polygon(ibound, row_coords_topo, col_coords_topo, 0)
plt.scatter(col_coords_topo,row_coords_topo,c='r')


array = change_array_within_polygon(ibound, row_coords_clay, col_coords_clay, 0)
plt.scatter(col_coords,row_coords,c='r')

for c in range(0,2):
    i = np.where(ibound[:,c,-1]==1)[0][-1]
    head = top_elev-i*layer_thickness
    #print('head {}'.format(head))
    
    while head<187:
        ibound[i,c,:] = -1
        #print(i,c)
        #initial_head[i,c,:] = head+0.5
        print(head)
        initial_head[i,c,:] = 187
        head+=layer_thickness
        i-=1


lay = np.where(ibound[:,int(1700/delr),-1])[0][0]
elv = lay_to_meter(lay)
head = elv

for c in range(int(1700/delr),int(2200/delr)):
    lay = np.where(ibound[:,c,-1]==1)[0][0]
    #print(lay,lay_to_meter(lay))
    ibound[lay,c,:] = -1
    initial_head[lay,c,:] = elv

# Your existing code
im = plt.imshow(ibound[:,:,2], origin='upper', aspect=1)
plt.axvline(x=meter_to_col(1700))# shoreline
plt.gcf().set_size_inches(15,10)

# Set the y-axis tick positions and labels
#plt.yticks([0, 10,20, 30,40, 50,60], ['190m (0L)','185m (10L)', '180m (20L)','175m (30L)', '170m (40L)', '165m (50L)','160m (60L)'])
# Add a horizontal colorbar

cbar = plt.colorbar(im, orientation='horizontal')
cbar.ax.tick_params(labelsize=16)
cbar.set_label('Colorbar Label', fontsize=16)

# Display the plot
plt.title('ibound')
plt.grid()
plt.show()


fig, ax = plt.subplots(figsize=(15, 10))
im = ax.imshow(initial_head[:, :, 0], origin='upper')

# Add a horizontal colorbar
cbar = plt.colorbar(im, orientation='horizontal')
cbar.ax.tick_params(labelsize=18)
plt.title('initial head')
plt.show()


# BAS LPF  PCG

In [ ]:

# Create BAS package
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=initial_head)

# Define hydraulic properties
horizontal_k = parms['hk'] # Horizontal hydraulic conductivity (m/day)
vertical_ka = 0.1   # Vertical hydraulic conductivity (m/day)
porosity = 0.25     # Porosity

# Create LPF package (Layer Property Flow package)
lpf = flopy.modflow.ModflowLpf(mf, hk=horizontal_k, 
                               vka=vertical_ka, laytyp=1, 
                               ipakcb=None,ss=0.0002,sy=0.33,
                               laywet=1,iwetit=1,wetfct=0.1,ihdwet=1)

# #ss=0.0002,sy=0.33
# upw = flopy.modflow.ModflowUpw(model, laytyp=1, hk=horizontal_k)

pcg = flopy.modflow.ModflowPcg(mf, hclose=1e-3,npcond=1,
                               relax=0.99,rclose=1e-3,mutpcg=0,nbpol=1,iprpcg=1)



# Heterogeneity

In [ ]:
# hk_mean = 2.3  # mean hydraulic conductivity
# hk_std = 2  # standard deviation of hydraulic conductivity
# hk= np.random.normal(hk_mean, hk_std, size=[nlay,nrow,ncol])
# hk[np.where(hk<0)]=0.001

# model.lpf.hk = hk


In [ ]:
# hk_mean = np.log10(2.3)  # mean hydraulic conductivity
# hk_std = 0.4  # standard deviation of hydraulic conductivity
# hk= np.random.normal(hk_mean, hk_std, size=[nlay,nrow,ncol])
# hk = 10**hk
# #hk[np.where(hk<0)]=0.001
# hk.mean()
# model.lpf.hk = hk

In [ ]:
# plt.hist(hk.flatten(),bins=np.linspace(0,30,50))
# plt.show()

# plt.imshow(hk[0,:,:])
# plt.show()

# DRN

In [ ]:
locate_ground_surface_lay(meter_to_row(160))
#drn_cell = [(0, 150, 28), -10, 100]

beach_drns = np.arange(1520,1600+1,delr).astype(int)
river_drns = np.arange(595,605+1,delr).astype(int)
#beach_drns = []

drn_list = []
drn_ys= list(beach_drns)+list(river_drns)

for y in drn_ys:
   
   row = meter_to_row(y)
   lay = locate_ground_surface_lay(row=row)
   elv = lay_to_meter(lay)
   print(row)
   #print(lay)
   for col in range(ncol):

      drn_list.append([lay, row, col, elv, 100])
      #drn_list.append([12*2, meter_to_row(600), col, 184.0-0.5, 100])
      
drn_cell = {0:drn_list} 

#drn_cell={0:[[31, 160, 0, 174.5-0.5, 0],[12, 60, 0, 184.0-0.5, 0]]}
drn = flopy.modflow.ModflowDrn(model=mf, stress_period_data=drn_cell,)

In [ ]:
[i for i in drn.stress_period_data.data[0] if i[2]==0] 

In [ ]:
lays_with_drn = np.unique([i[0] for i in drn.stress_period_data.data[0]])
for lay in lays_with_drn:
    pmv = flopy.plot.PlotMapView(model=mf, layer=lay)
    pmv.plot_bc('DRN',color='r')

# RCH

In [ ]:
rch_dict = {}

rch_array = np.full(( nrow, ncol), 0.00013, dtype=float)
rch_dict[0] = rch_array

rch_array = np.full(( nrow, ncol), 0.00013, dtype=float)
#rch_array[int(nrow/2),int(ncol/2)] = 0.51/delr/delc
#rch_array[int(nrow/2),int(ncol/3)] = 0.51/delr/delc

rch_array[int((1700-450)/delr),int(ncol/2)] = 0.51/delr/delc
rch_array[int((1700-450)/delr),int(ncol/3)] = 0.51/delr/delc

rch_dict[1] = rch_array 
rch = flopy.modflow.ModflowRch(model = mf, rech = rch_dict)

In [ ]:
# shoreline at 1700m , septic at 1700-60

# OC

In [ ]:
# OC
spd = {}
for kper in range(nper):
    for kstp in range(1):
        spd[(kper, kstp)] = [
            "save head",
            "save drawdown",
            "save budget",
            "print head",
            "print budget",
        ]

oc = flopy.modflow.ModflowOc(
    mf,
    stress_period_data= spd,
    compact=True
    ,extension=['oc', 'hds', 'ddn', 'cbc', 'ibo']
)

#oc.reset_budgetunit(budgetunit=1053, fname='test.cbc')
oc.reset_budgetunit(budgetunit=53, fname='example_model.cbc')

In [ ]:
lmt = flopy.modflow.ModflowLmt(mf, output_file_name='mt3d_link.ftl')

In [ ]:
# Write the MODFLOW model input files
mf.write_input()

In [ ]:
# Run the MODFLOW model
success, buff = mf.run_model()

# ---MT3DMS---

In [ ]:
mt = flopy.mt3d.Mt3dms(modflowmodel=mf, modelname='mt', exe_name='mt3dms.exe' ,ftlfilename='mt3d_link.ftl',model_ws=model_ws)

# ICBUND

In [ ]:
icbund = ibound.copy()
cc = np.where(icbund==-1)
mask = cc[1]>2200/2/delr

cc2 = (cc[0][mask],cc[1][mask],cc[2][mask])
icbund[cc2] = 1
plt.imshow(icbund[:,:,0])

# BTN ICBOUND 

In [ ]:
#icbund = np.full((nlay, nrow, ncol),0, dtype=float)
#
btn = flopy.mt3d.Mt3dBtn(
    mt,
    nprs=5,
    timprs=np.cumsum([200]*7),
    prsity=0.25,
    sconc= 0.01, # starting concentration
    ifmtcn=0, chkmas=False, nprobs=10, nprmas=10,
    dt0=5,  # The user-specified initial 
    ttsmult=1.4,
    mxstrn = 50000,
    icbund = icbund
)


In [ ]:
adv = flopy.mt3d.Mt3dAdv(mt, mixelm=0,percel=0.4,mxpart=50000000,nadvfd=0)
dsp = flopy.mt3d.Mt3dDsp(mt, al=parms['al'], trpt=0.1, 
                        trpv=0.1, 
                        dmcoef=3.7E-10 )
gcg = flopy.mt3d.Mt3dGcg(mt, iter1=50, mxiter=1, isolve=2, cclose= 1e-5)

# RCH

In [ ]:
# crch_array =  np.full(( nrow, ncol), 0.1, dtype=float)
# crch_dict = {}
# crch_dict[0] = crch_array
# ssm = flopy.mt3d.Mt3dSsm(model, crch=crch_dict)

# rch_dict = {}
# rch_array = np.full(( nrow, ncol), 0.00013, dtype=float)
# rch_dict[0] = rch_array
# rch = flopy.modflow.ModflowRch(model = model, rech = rch_dict)


# ## [row, col, rch(m3), crch, start, end]
# rch_data = [[1100,1100,0.51,5000,1,20]]


In [ ]:
crch_array =  np.full(( nrow, ncol), 0.1, dtype=float)
crch_dict = {}
crch_dict[0] = crch_array

crch_array =  np.full(( nrow, ncol), 0.1, dtype=float)

# crch_array[int(nrow/2),int(ncol/2)] =5000
# crch_array[int(nrow/2),int(ncol/3)] =5000
crch_array[int((1700-450)/delr),int(ncol/3)] =5000
crch_array[int((1700-450)/delr),int(ncol/2)] =5000
spetic_locs = [[int((1700-450)/delr),int(ncol/3)], [int((1700-450)/delr),int(ncol/2)]]

crch_dict = {}
crch_dict[1] = crch_array


crch_array =  np.full(( nrow, ncol), 0.1, dtype=float)
crch_dict[69] = crch_array

ssm = flopy.mt3d.Mt3dSsm(mt, crch=crch_dict)

In [ ]:
im = plt.imshow(ibound[:, :, 0], origin='upper')

drns0 = np.unique([i for i in drn.stress_period_data.data[0] if i[2]==0])
for i in drns0:
    plt.scatter(i[1],i[0],c='cyan',s=5,marker='s')


for i in spetic_locs:

    plt.scatter([i[0],i[0],i[0],i[0]],[1,2,3,4],c='red',s=6,marker='s')
    




plt.gcf().set_size_inches(15,10)
plt.show()

# RCT

In [ ]:
rct = flopy.mt3d.Mt3dRct(model=mt, isothm=3,sp1=parms['sp1'],sp2=0.0001,igetsc=0,rhob= 1.65E+09)

# Write and Run

In [ ]:

# Write MODFLOW input files
mt.write_input()

In [ ]:
open_file(model_ws)

In [36]:
# Run the MODFLOW model
mt.run_model()

 Outer Iter.  1  Inner Iter.  4:  Max. DC =  0.1338E-03  [K,I,J]   52  252  147
 Outer Iter.  1  Inner Iter.  5:  Max. DC =  0.2664E-04  [K,I,J]   52  250  147
 Outer Iter.  1  Inner Iter.  6:  Max. DC =  0.2265E-05  [K,I,J]   52  251  147
 Transport Step:   11   Step Size:   15.93     Total Elapsed Time:   3650.0    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =  0.9027E-03  [K,I,J]   54  253  221
 Outer Iter.  1  Inner Iter.  2:  Max. DC =  0.1380E-03  [K,I,J]   52  252  147
 Outer Iter.  1  Inner Iter.  3:  Max. DC =  0.7510E-05  [K,I,J]   52  252  147

 Program completed.   Total CPU time:  013 minutes 54.734 seconds


(False, [])

In [37]:
model_ws

'C:\\Users\\swan47\\AppData\\Local\\Temp\\_T1213133434_wasaga_a4inkdh2'